In [ ]:
import cv2 as cv
import numpy as np

# Function untuk deteksi bola ping - pong dan membuat bounding box dari bola ping - pong tersebut
def detect_pingpong_ball(frame, lower_hsv, upper_hsv):
    # Frame kamera dikonversi ke HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)

    # Pembuatan masking dari nilai range HSV yang sudah ditentukan
    mask = cv.inRange(hsv, lower_hsv, upper_hsv)

    # Pencarian outline atau contour dari objek yang terfilter pada mask 
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Pemfilteran contour yang terdeteksi dengan hanya menggunakan contour yang paling besar
    if contours:
        largest_contour = max(contours, key=cv.contourArea)
        if cv.contourArea(largest_contour) > 500:  
            # Pembuatan bounding box pada contour yang paling besar
            x, y, w, h = cv.boundingRect(largest_contour)
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  

    return frame, mask

# Function untuk membuat trackbar HSV agar bisa dinamik / bisa diubah pada saat program berjalan
def nothing(x):
    pass

# Pembuatan window untuk trackbar beserta parameter HSV
cv.namedWindow("Trackbars")
cv.createTrackbar("HMin", "Trackbars", 0, 179, nothing)
cv.createTrackbar("SMin", "Trackbars", 0, 255, nothing)
cv.createTrackbar("VMin", "Trackbars", 0, 255, nothing)
cv.createTrackbar("HMax", "Trackbars", 0, 179, nothing)
cv.createTrackbar("SMax", "Trackbars", 0, 255, nothing)
cv.createTrackbar("VMax", "Trackbars", 0, 255, nothing)

# Nilai default HSV ditentukan berdasarkan hasil filter HSV dari bola ping - pong
cv.setTrackbarPos("HMin", "Trackbars", 9)
cv.setTrackbarPos("SMin", "Trackbars", 83)
cv.setTrackbarPos("VMin", "Trackbars", 142)
cv.setTrackbarPos("HMax", "Trackbars", 22)
cv.setTrackbarPos("SMax", "Trackbars", 220)
cv.setTrackbarPos("VMax", "Trackbars", 255)

cap = cv.VideoCapture(1)  

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Nilai posisi dari trackbar untuk setiap parameter diupdate disini
    h_min = cv.getTrackbarPos("HMin", "Trackbars")
    s_min = cv.getTrackbarPos("SMin", "Trackbars")
    v_min = cv.getTrackbarPos("VMin", "Trackbars")
    h_max = cv.getTrackbarPos("HMax", "Trackbars")
    s_max = cv.getTrackbarPos("SMax", "Trackbars")
    v_max = cv.getTrackbarPos("VMax", "Trackbars")

    # Pembuatan nilai minimum dan maksimum untuk HSV
    lower_hsv = np.array([h_min, s_min, v_min])
    upper_hsv = np.array([h_max, s_max, v_max])

    # Deteksi bola ping - pong dan pembuatan bounding box
    frame_with_box, mask = detect_pingpong_ball(frame, lower_hsv, upper_hsv)

    # Penampilan frame bola ping - pong beserta hasil maskingnya
    cv.imshow('Ping Pong Ball Detection', frame_with_box)
    cv.imshow('Mask', mask)

    # Pencet tombol q untuk keluar dari program
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()